In [1]:
import os
import sys
import pandas as pd
from datetime import datetime
import sqlalchemy 

# Adicionando caminhos dos módulos
sys.path.insert(0, r'C:\Scripts\modules_azure\database')
sys.path.insert(0, r'C:\Scripts\modules_azure\parameters')

from connection_azure import Connect
from azure_loader import AzureLoader
from parametros import Parametros

SCHEMA_DEFAULT = "dbo"
CAMINHO_RELATORIOS = r"C:\Scripts\relatórios" # Raiz para varredura

In [2]:
try:
    # Carrega tabela base_btg atual do Azure (para referência futura se precisar)
    base_atual_db = AzureLoader.ler_tabela("base_btg", schema=SCHEMA_DEFAULT)
    base_atual_db['Conta'] = base_atual_db['Conta'].astype(str).str.strip()
    
    # Carrega Nomes dos Clientes
    nomes_clientes = AzureLoader.ler_tabela("nomes_clientes", schema=SCHEMA_DEFAULT)
    nomes_clientes['Conta'] = nomes_clientes['Conta'].astype(str).str.strip()
    
    # Carrega Entradas Consolidadas para saber onde parou
    entradas_consolidado = AzureLoader.ler_tabela("Entradas_e_saidas_consolidado", schema=SCHEMA_DEFAULT)
    
except Exception as e:
    print(f"Erro na carga inicial: {e}")
    sys.exit()

[AzureLoader] Lendo: dbo.base_btg...
[AzureLoader] Lendo: dbo.nomes_clientes...
[AzureLoader] Lendo: dbo.Entradas_e_saidas_consolidado...


In [3]:
pastas_dias = []
if os.path.exists(CAMINHO_RELATORIOS):
    for item in os.listdir(CAMINHO_RELATORIOS):
        caminho_completo = os.path.join(CAMINHO_RELATORIOS, item)
        if os.path.isdir(caminho_completo):
            try:
                datetime.strptime(item, "%d-%m-%Y")
                pastas_dias.append({"dia": item})
            except ValueError:
                continue

pastas_dias.sort(key=lambda x: datetime.strptime(x["dia"], "%d-%m-%Y"))

# Define ponto de corte (última data processada)
if not entradas_consolidado.empty:
    entradas_consolidado['Mês de entrada/saída'] = pd.to_datetime(entradas_consolidado['Mês de entrada/saída'])
    max_data_banco = entradas_consolidado['Mês de entrada/saída'].max()
else:
    max_data_banco = pd.to_datetime('2020-01-01') # Data antiga padrão

print(f"   -> Última data no banco: {max_data_banco.date()}")

# Filtra pastas novas
pastas_para_processar = [
    p for p in pastas_dias 
    if pd.to_datetime(p['dia'], format='%d-%m-%Y') > max_data_banco
]

# Tenta pegar a pasta do dia exato da última atualização para servir de base anterior (D-1)
pasta_base_anterior = next((p for p in pastas_dias if pd.to_datetime(p['dia'], format='%d-%m-%Y') == max_data_banco), None)

if pasta_base_anterior and pastas_para_processar:
    pastas_para_processar.insert(0, pasta_base_anterior)
elif not pasta_base_anterior and pastas_para_processar:
    print("   -> Aviso: Pasta do dia anterior não encontrada. O primeiro dia da lista será comparado com uma base vazia ou pulado.")

   -> Última data no banco: 2026-02-02


In [4]:
lista_entrada_saida = []

if len(pastas_para_processar) < 2:
    print("   -> Banco já está atualizado. Nenhuma nova comparação necessária.")
else:
    print(f"   -> Processando {len(pastas_para_processar)-1} intervalos...")

    for passado, presente in zip(pastas_para_processar[:-1], pastas_para_processar[1:]):
        dia_passado_str = passado['dia']
        dia_presente_str = presente['dia']
        
        try:
            # --- Leitura Base Passada ---
            caminho_passado = os.path.join(CAMINHO_RELATORIOS, dia_passado_str)
            # Pega primeira subpasta (horário)
            sub_passado = os.listdir(caminho_passado)[0]
            arq_passado = os.path.join(caminho_passado, sub_passado, "base_btg.xlsx")
            
            try:
                base_ant = pd.read_excel(arq_passado, header=2)
                if 'Conta' not in base_ant.columns: base_ant = pd.read_excel(arq_passado)
            except:
                base_ant = pd.read_excel(arq_passado)
            
            base_ant['Conta'] = base_ant['Conta'].astype(str).str.strip()

            # --- Leitura Base Presente ---
            caminho_presente = os.path.join(CAMINHO_RELATORIOS, dia_presente_str)
            sub_presente = os.listdir(caminho_presente)[0]
            arq_presente = os.path.join(caminho_presente, sub_presente, "base_btg.xlsx")
            
            try:
                base_atual = pd.read_excel(arq_presente, header=2)
                if 'Conta' not in base_atual.columns: base_atual = pd.read_excel(arq_presente)
            except:
                base_atual = pd.read_excel(arq_presente)
            
            base_atual['Conta'] = base_atual['Conta'].astype(str).str.strip()

            # --- Comparação (Churn e Captação) ---
            
            # Saiu: Estava na anterior e NÃO está na atual
            saiu = base_ant[~base_ant['Conta'].isin(base_atual['Conta'])].copy()
            saiu['Situação'] = "Saiu"
            saiu['Mês de entrada/saída'] = dia_presente_str
            
            # Entrou: NÃO estava na anterior e ESTÁ na atual
            entrou = base_atual[~base_atual['Conta'].isin(base_ant['Conta'])].copy()
            entrou['Situação'] = "Entrou"
            entrou['Mês de entrada/saída'] = dia_presente_str
            
            if 'Faixa Cliente' in entrou.columns:
                entrou.drop("Faixa Cliente", axis=1, inplace=True)

            # Consolidar do dia
            mov_dia = pd.concat([saiu, entrou], axis=0)
            
            cols_desejadas = ['Conta', 'Assessor', "PL Total", "PL Declarado", "Faixa Cliente", 'Data Vínculo', 'Situação', "Mês de entrada/saída"]
            cols_existentes = [c for c in cols_desejadas if c in mov_dia.columns]
            mov_dia = mov_dia[cols_existentes]
            
            # Remove contas técnicas ignoradas
            contas_ignorar = ['1983816', '1106619']
            mov_dia = mov_dia[~mov_dia['Conta'].isin(contas_ignorar)]
            
            # Adiciona nomes
            mov_dia = pd.merge(mov_dia, nomes_clientes, on='Conta', how='left')
            
            if not mov_dia.empty:
                lista_entrada_saida.append(mov_dia)
                print(f"      [{dia_presente_str}] OK: {len(mov_dia)} movimentações.")
            
        except Exception as e:
            print(f"      [{dia_presente_str}] ERRO: {e}")

   -> Processando 4 intervalos...
      [03-02-2026] OK: 1 movimentações.
      [04-02-2026] OK: 93 movimentações.
      [05-02-2026] OK: 83 movimentações.
      [06-02-2026] OK: 3 movimentações.


In [5]:
if not lista_entrada_saida:
    print("   -> Nenhuma movimentação nova para salvar.")
    # table = pd.DataFrame(...) # Se quiser criar vazio
    sys.exit()

table = pd.concat(lista_entrada_saida, ignore_index=True)
table = table.drop_duplicates(subset=['Conta'], keep='last')

# Tratamentos de Texto
table['Assessor'] = table['Assessor'].astype(str).str.upper()
table.loc[table['Assessor'].isin(["RODRIGO DE MELLO D?ELIA", "RODRIGO DE MELLO DELIA"]), "Assessor"] = "RODRIGO DE MELLO D’ELIA"

# Ajustes Manuais
table.loc[table['Conta'].isin(['590732', '299305']), 'Assessor'] = 'JOSE AUGUSTO ALVES DE PAULA FILHO'

# Converte data para datetime para subir pro banco
table['Mês de entrada/saída'] = pd.to_datetime(table['Mês de entrada/saída'], format="%d-%m-%Y")

In [6]:
lista_entrada_saida = []
diretorio_base = r"C:\Scripts\relatórios"

if len(pastas_para_processar) < 2:
    print("Não há dias suficientes para comparação (Banco já está atualizado).")

else:
    print(f"Iniciando processamento de {len(pastas_para_processar)-1} novos dias...")

    # O zip garante que comparamos o dia atual (presente) com o dia anterior (passado)
    for passado, presente in zip(pastas_para_processar[:-1], pastas_para_processar[1:]):
        
        dia_passado_str = passado['dia']
        dia_presente_str = presente['dia']
        
        try:
            # --- BUSCA BASE ANTERIOR ---
            caminho_dia_passado = os.path.join(diretorio_base, dia_passado_str)
            subpasta_passado = os.listdir(caminho_dia_passado)[0]
            arquivo_passado = os.path.join(caminho_dia_passado, subpasta_passado, "base_btg.xlsx")
            
            try:
                base_anterior = pd.read_excel(arquivo_passado, header=2)
                if 'Conta' not in base_anterior.columns:
                    base_anterior = pd.read_excel(arquivo_passado)
            except:
                base_anterior = pd.read_excel(arquivo_passado)
            
            base_anterior['Conta'] = base_anterior['Conta'].astype(str)

            # --- BUSCA BASE ATUAL ---
            caminho_dia_presente = os.path.join(diretorio_base, dia_presente_str)
            subpasta_presente = os.listdir(caminho_dia_presente)[0]
            arquivo_atual = os.path.join(caminho_dia_presente, subpasta_presente, "base_btg.xlsx")
            
            try:
                base_atual = pd.read_excel(arquivo_atual, header=2)
                if 'Conta' not in base_atual.columns:
                    base_atual = pd.read_excel(arquivo_atual)
            except:
                base_atual = pd.read_excel(arquivo_atual)
                
            base_atual['Conta'] = base_atual['Conta'].astype(str)

            # --- LÓGICA DE ENTRADA E SAÍDA ---
            # Clientes que estavam na anterior e NÃO estão na atual (SAÍRAM)
            # Nota: O Mês de saída deve ser o do dia PRESENTE (data do evento), mas seu código original usava passado. 
            # Mantive a lógica original, mas verifique se não prefere `dia_presente_str`.
            clientes_sairam = base_anterior[~base_anterior['Conta'].isin(base_atual['Conta'])].copy()
            clientes_sairam['Situação'] = "Saiu"
            clientes_sairam['Mês de entrada/saída'] = dia_presente_str # Ajustado para data do evento

            # Clientes que NÃO estavam na anterior e estão na atual (ENTRARAM)
            clientes_entraram = base_atual[~base_atual['Conta'].isin(base_anterior['Conta'])].copy()
            clientes_entraram['Situação'] = "Entrou"
            clientes_entraram['Mês de entrada/saída'] = dia_presente_str # Ajustado para data do evento
            
            if 'Faixa Cliente' in clientes_entraram.columns:
                clientes_entraram.drop("Faixa Cliente", axis=1, inplace=True)
            
            # Consolida
            entradas_saidas = pd.concat([clientes_sairam, clientes_entraram], axis=0)
            
            colunas_alvo = ['Conta', 'Assessor', "PL Total", "PL Declarado", "Faixa Cliente", 'Data Vínculo', 'Situação', "Mês de entrada/saída"]
            colunas_existentes = [c for c in colunas_alvo if c in entradas_saidas.columns]
            
            entradas_saidas = entradas_saidas[colunas_existentes]
            
            entradas_saidas = entradas_saidas[~entradas_saidas['Conta'].isin(['1983816', '1106619'])]
            entradas_saidas = pd.merge(entradas_saidas, nomes_clientes, on='Conta', how='left')

            if not entradas_saidas.empty:
                lista_entrada_saida.append(entradas_saidas)
                print(f"[OK] Comparado: {dia_passado_str} -> {dia_presente_str} (Gerou {len(entradas_saidas)} linhas)")
            else:
                print(f"[OK] Comparado: {dia_passado_str} -> {dia_presente_str} (Nenhuma alteração encontrada)")

        except Exception as e:
            print(f"[ERRO] Falha ao comparar {dia_passado_str} com {dia_presente_str}: {e}")

# --- CONSOLIDAÇÃO FINAL COM PROTEÇÃO ---
if lista_entrada_saida:
    entradas_saidas_consolidado = pd.concat(lista_entrada_saida, ignore_index=True)
    table = entradas_saidas_consolidado.copy()
    print(f"{len(table)} novas movimentações identificadas.")
else:
    print("Lista Vazia.")
    # Cria um DataFrame vazio com as colunas esperadas para não quebrar códigos futuros
    table = pd.DataFrame(columns=['Conta', 'Situação', 'Mês de entrada/saída'])

Iniciando processamento de 4 novos dias...
[OK] Comparado: 02-02-2026 -> 03-02-2026 (Gerou 1 linhas)
[OK] Comparado: 03-02-2026 -> 04-02-2026 (Gerou 93 linhas)
[OK] Comparado: 04-02-2026 -> 05-02-2026 (Gerou 83 linhas)
[OK] Comparado: 05-02-2026 -> 06-02-2026 (Gerou 3 linhas)
180 novas movimentações identificadas.


In [8]:
# Filtra apenas quem "Entrou" (Novas Contas)
novas_contas = table[table['Situação'] == "Entrou"].copy()

if not novas_contas.empty:

    # 1. Renomeia colunas básicas
    novas_contas.rename(columns={
        'Mês de entrada/saída': 'DATA',
        'Conta': 'CONTA',
        'PL Total': 'CAPTAÇÃO'
    }, inplace=True)
    
    # 2. Cria colunas adicionais para padronizar
    novas_contas['DESCRIÇÃO'] = "Abertura de Conta"
    novas_contas['MERCADO'] = "Abertura de Conta"
    novas_contas['TIPO DE CAPTACAO'] = "Abertura de Conta"
    novas_contas['Situacao'] = "Ativo"
    novas_contas['TIPO'] = "Abertura"
    
    # 3. Limpeza para a tabela Historico
    cols_captacao = ['DATA', 'CONTA', 'CAPTAÇÃO', 'Assessor', 'Nome', 
                     'DESCRIÇÃO', 'MERCADO', 'TIPO DE CAPTACAO', 'Situacao', 'TIPO']
    
    cols_finais_hist = [c for c in cols_captacao if c in novas_contas.columns]
    df_upload_captacao = novas_contas[cols_finais_hist].copy()
    
    # 4. Tratamento de Tipos
    df_upload_captacao['CAPTAÇÃO'] = df_upload_captacao['CAPTAÇÃO'].fillna(0)
    df_upload_captacao['CONTA'] = df_upload_captacao['CONTA'].astype(str).str.strip()
    
    # 5. Upload para CAPTACAO_HISTORICO
    print("   -> Inserindo em captacao_historico...")
    AzureLoader.enviar_df(
        df=df_upload_captacao,
        nome_tabela="captacao_historico",
        if_exists="append", 
        schema=SCHEMA_DEFAULT
    )
    
    df_migracao = df_upload_captacao.copy()
    df_migracao['Considerar'] = "SIM"
    
    # Definição ESTRITA das colunas que sua tabela aceita
    cols_migracao_schema = ['DATA', 'CONTA', 'Assessor', 'CAPTAÇÃO', 'Considerar']
    
    # Seleciona apenas as colunas que existem no DataFrame e estão na lista permitida
    df_migracao_final = df_migracao[cols_migracao_schema].copy()

    print("   -> Inserindo em migracoes_btg...")
    AzureLoader.enviar_df(
        df=df_migracao_final,
        nome_tabela="migracoes_btg",
        if_exists="append",
        schema=SCHEMA_DEFAULT
    )

else:
    print("   -> Nenhuma conta nova para registrar.")

print("\n--- Script Finalizado com Sucesso ---")

   -> Inserindo em captacao_historico...
[AzureLoader] Subindo tabela: dbo.captacao_historico (86 linhas)...
[AzureLoader] Chunksize calculado: 209 linhas/lote (Colunas: 10).
[AzureLoader] Concluido: captacao_historico atualizada.
   -> Inserindo em migracoes_btg...
[AzureLoader] Subindo tabela: dbo.migracoes_btg (86 linhas)...
[AzureLoader] Chunksize calculado: 418 linhas/lote (Colunas: 5).
[AzureLoader] Concluido: migracoes_btg atualizada.

--- Script Finalizado com Sucesso ---
